In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv('/content/drive/MyDrive/ESAA/movies_train.csv')
test = pd.read_csv('/content/drive/MyDrive/ESAA/movies_test.csv')
sub = pd.read_csv('/content/drive/MyDrive/ESAA/submission.csv')

Mounted at /content/drive


In [ ]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [ ]:
test.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3
2,하모니,CJ 엔터테인먼트,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7
3,의형제,(주)쇼박스,액션,2010-02-04,116,15세 관람가,장훈,6.913420e+05,2,408,2
4,평행 이론,CJ 엔터테인먼트,공포,2010-02-18,110,15세 관람가,권호영,3.173800e+04,1,380,1


In [ ]:
sub.head()

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0


#**전처리**

##**결측치 처리**

In [ ]:
# train 결측치 확인
print(train.isnull().sum())

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    330
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
dtype: int64


In [ ]:
# test 결측치 확인
print(test.isnull().sum())

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    136
dir_prev_num        0
num_staff           0
num_actor           0
dtype: int64


In [ ]:
# dir_prev_bfunm 결측치를 0으로 변경
train['dir_prev_bfnum'] = train['dir_prev_bfnum'].fillna(0)
test['dir_prev_bfnum'] = test['dir_prev_bfnum'].fillna(0)

In [ ]:
# 결측치 확인
print(train.isnull().sum(), '\n')
print(test.isnull().sum())

title             0
distributor       0
genre             0
release_time      0
time              0
screening_rat     0
director          0
dir_prev_bfnum    0
dir_prev_num      0
num_staff         0
num_actor         0
box_off_num       0
dtype: int64 

title             0
distributor       0
genre             0
release_time      0
time              0
screening_rat     0
director          0
dir_prev_bfnum    0
dir_prev_num      0
num_staff         0
num_actor         0
dtype: int64


##**날짜형 변수 처리**

In [ ]:
print(train.columns)
print(test.columns)


Index(['title', 'distributor', 'genre', 'release_time', 'time',
       'screening_rat', 'director', 'dir_prev_bfnum', 'dir_prev_num',
       'num_staff', 'num_actor', 'box_off_num'],
      dtype='object')
Index(['title', 'distributor', 'genre', 'release_time', 'time',
       'screening_rat', 'director', 'dir_prev_bfnum', 'dir_prev_num',
       'num_staff', 'num_actor'],
      dtype='object')


In [ ]:
# 개봉일을 연도, 월로 분리

# 문자열을 datetime으로 변환
train['release_time'] = pd.to_datetime(train['release_time'])
test['release_time'] = pd.to_datetime(test['release_time'])

# release_year, release_month
train['release_year'] = train['release_time'].dt.year
train['release_month'] = train['release_time'].dt.month
test['release_year'] = test['release_time'].dt.year
test['release_month'] = test['release_time'].dt.month

# release time 제거
train = train.drop(['release_time'], axis=1)
test = test.drop(['release_time'], axis=1)

##**이상치 로그변환**

In [ ]:
#이상치 확인
Q1 = train['box_off_num'].quantile(0.25)
Q3 = train['box_off_num'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = train[(train['box_off_num'] < lower) | (train['box_off_num'] > upper)]
out_num = outliers['box_off_num']

print(f'관객수 이상치 개수: {len(out_num)}')

관객수 이상치 개수: 97


In [ ]:
# 관객수 이상치 로그변환
train['box_off_num_log'] = np.log1p(train['box_off_num'])

##**배급사명 통일**

In [ ]:
# 배급사값 확인
print(sorted(train['distributor'].unique()))
print(sorted(test['distributor'].unique()))

['(주) 브릿지웍스', '(주) 액티버스엔터테인먼트', '(주) 영화사조제', '(주) 영화제작전원사', '(주) 케이알씨지', '(주)JK필름', '(주)NEW', '(주)고앤고 필름', '(주)노버스엔터테인먼트', '(주)대명문화공장', '(주)더픽쳐스', '(주)두타연', '(주)드림팩트엔터테인먼트', '(주)랠리버튼', '(주)레인보우 팩토리', '(주)로드하우스', '(주)리틀빅픽처스', '(주)마운틴픽쳐스', '(주)마인스 엔터테인먼트', '(주)마인스엔터테인먼트', '(주)미디어데이', '(주)쇼박스', '(주)스마일이엔티', '(주)스톰픽쳐스코리아', '(주)시네마달', '(주)씨엠닉스', '(주)아우라픽처스', '(주)아이필름코퍼레이션', '(주)에이원 엔터테인먼트', '(주)엣나인필름', '(주)예지림 엔터테인먼트', '(주)온비즈넷', '(주)유비네트워크', '(주)유비콘텐츠', '(주)이스트스카이필름', '(주)콘텐츠 윙', '(주)키노아이', '(주)팝 파트너스', '(주)팝엔터테인먼트', '(주)패뷸러스', '(주)패스파인더씨앤씨', '(주)프레인글로벌', '(주)피터팬픽쳐스', '(주)휘엔터테인먼트', 'BoXoo 엔터테인먼트', 'CGV 무비꼴라쥬', 'CGV 아트하우스', 'CGV아트하우스', 'CJ CGV', 'CJ E&M Pictures', 'CJ E&M 영화부문', 'CJ 엔터테인먼트', 'KBS미디어(주)', 'KT', 'KT&G 상상마당', 'M2픽처스', 'NEW', 'OAL(올)', 'SBS콘텐츠허브', 'SK텔레콤(주)', 'SK플래닛', 'THE 픽쳐스', 'tvN', 'ysfilm', '건시네마', '고구마공작소', '골든타이드픽처스', '김기덕 필름', '나우콘텐츠', '나이너스엔터테인먼트(주)', '노바엔터테인먼트', '더 피플', '더픽쳐스/(주)마운틴픽쳐스', '동국대학교 충무로영상제작센터', '두 엔터테인먼트', '드림로드', '드림팩트 엔터테인먼트', '디 씨드', '디마엔터테인먼트', '롤러코스터 

In [ ]:
train['distributor'].value_counts()

,count
distributor,
CJ 엔터테인먼트,54
롯데엔터테인먼트,52
(주)NEW,30
(주)마운틴픽쳐스,29
(주)쇼박스,26
...,...
영화사 廊,1
크리에이티브컴즈(주),1
ysfilm,1


In [ ]:
test['distributor'].value_counts()

,count
distributor,
CJ 엔터테인먼트,31
롯데엔터테인먼트,24
(주)마운틴픽쳐스,14
(주)쇼박스,12
골든타이드픽처스,12
...,...
(주)스톰픽쳐스코리아,1
(주)다세포클럽,1
디씨드,1


In [ ]:
# 배급사명 통일
dist_map = {
    'CJ 엔터테인먼트': 'CJE&M Movie',
    'CJ E&M 영화부문': 'CJE&M Movie',
    'CJ E&M Pictures': 'CJE&M Movie',

    '(주)쇼박스': '쇼박스',
    '쇼박스(주)미디어플렉스': '쇼박스',

    '(주)NEW': 'NEW',

    '리틀빅픽처스': '리틀빅픽처스',
    '리틀빅픽쳐스': '리틀빅픽처스',

    '스폰지': '스폰지이엔티',
    '스폰지이엔티': '스폰지이엔티',

    '시네마 달': '시네마달',

    '드림팩트 엔터테인먼트': '드림팩트엔터테인먼트',
    '(주)드림팩트엔터테인먼트': '드림팩트엔터테인먼트',

    '(주)마운틴픽쳐스': '마운틴픽처스',
    '마운틴 픽처스': '마운틴픽처스',
    '마운틴픽쳐스': '마운틴픽처스',

    '(주)에이원 엔터테인먼트': '에이원엔터테인먼트',
    '에이원 엔터테인먼트': '에이원엔터테인먼트',

    '롯데쇼핑(주)롯데엔터테인먼트': '롯데엔터테인먼트',

    'CGV 아트하우스': 'CGV아트하우스',
    'CGV아트하우스': 'CGV아트하우스',
    'CGV 무비꼴라쥬': 'CGV아트하우스',
    '무비꼴라쟈쥬': 'CGV아트하우스',

    '싸이더스': '싸이더스FNH',
    '싸이더스 FNH': '싸이더스FNH',
    '싸이더스FNH': '싸이더스FNH',

    '(주) 마인스 엔터테인먼트': '(주)마인스엔터테인먼트',
}

# 매핑 적용
train['distributor'] = train['distributor'].replace(dist_map)
test['distributor'] = test['distributor'].replace(dist_map)

# 확인
print(sorted(train['distributor'].unique()))
print(sorted(test['distributor'].unique()))

['(주) 브릿지웍스', '(주) 액티버스엔터테인먼트', '(주) 영화사조제', '(주) 영화제작전원사', '(주) 케이알씨지', '(주)JK필름', '(주)고앤고 필름', '(주)노버스엔터테인먼트', '(주)대명문화공장', '(주)더픽쳐스', '(주)두타연', '(주)랠리버튼', '(주)레인보우 팩토리', '(주)로드하우스', '(주)리틀빅픽처스', '(주)마인스 엔터테인먼트', '(주)마인스엔터테인먼트', '(주)미디어데이', '(주)스마일이엔티', '(주)스톰픽쳐스코리아', '(주)시네마달', '(주)씨엠닉스', '(주)아우라픽처스', '(주)아이필름코퍼레이션', '(주)엣나인필름', '(주)예지림 엔터테인먼트', '(주)온비즈넷', '(주)유비네트워크', '(주)유비콘텐츠', '(주)이스트스카이필름', '(주)콘텐츠 윙', '(주)키노아이', '(주)팝 파트너스', '(주)팝엔터테인먼트', '(주)패뷸러스', '(주)패스파인더씨앤씨', '(주)프레인글로벌', '(주)피터팬픽쳐스', '(주)휘엔터테인먼트', 'BoXoo 엔터테인먼트', 'CGV아트하우스', 'CJ CGV', 'CJE&M Movie', 'KBS미디어(주)', 'KT', 'KT&G 상상마당', 'M2픽처스', 'NEW', 'OAL(올)', 'SBS콘텐츠허브', 'SK텔레콤(주)', 'SK플래닛', 'THE 픽쳐스', 'tvN', 'ysfilm', '건시네마', '고구마공작소', '골든타이드픽처스', '김기덕 필름', '나우콘텐츠', '나이너스엔터테인먼트(주)', '노바엔터테인먼트', '더 피플', '더픽쳐스/(주)마운틴픽쳐스', '동국대학교 충무로영상제작센터', '두 엔터테인먼트', '드림로드', '드림팩트엔터테인먼트', '디 씨드', '디마엔터테인먼트', '롤러코스터 프로덕션', '롯데엔터테인먼트', '리틀빅픽처스', '마노엔터테인먼트', '마법사필름', '마운틴픽처스', '머스트 씨 무비', '메가박스(주)플러스엠', '무비꼴라쥬', '미라클 필름', '백두대간', '보람엔터테인먼트', 

In [ ]:
train['distributor'].value_counts()

,count
distributor,
CJE&M Movie,59
롯데엔터테인먼트,52
NEW,38
마운틴픽처스,32
쇼박스,28
...,...
영화사 廊,1
크리에이티브컴즈(주),1
ysfilm,1


In [ ]:
test['distributor'].value_counts()

,count
distributor,
CJE&M Movie,32
롯데엔터테인먼트,25
마운틴픽처스,15
쇼박스,14
골든타이드픽처스,12
...,...
(주)스톰픽쳐스코리아,1
(주)다세포클럽,1
디씨드,1


##**라벨인코딩**

In [ ]:
cols = ['distributor', 'genre', 'screening_rat', 'director']

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in cols:
    le = LabelEncoder()

    # train + test 묶어서 fit
    combined = pd.concat([train[col], test[col]]).astype(str)

    le.fit(combined)

    # transform 각각 적용
    train[col + '_le'] = le.transform(train[col].astype(str))
    test[col + '_le'] = le.transform(test[col].astype(str))

    label_encoders[col] = le

In [ ]:
train.head()

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,release_year,release_month,box_off_num_log,distributor_le,genre_le,screening_rat_le,director_le
0,개들의 전쟁,롯데엔터테인먼트,액션,96,청소년 관람불가,조병옥,0.00,0,91,2,23398,2012,11,10.060449,87,10,3,517
1,내부자들,쇼박스,느와르,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,2015,11,15.771725,110,2,3,327
2,은밀하게 위대하게,쇼박스,액션,123,15세 관람가,장철수,220775.25,4,343,4,6959083,2013,6,15.755558,110,10,1,467
3,나는 공무원이다,NEW,코미디,101,전체 관람가,구자홍,23894.00,2,20,6,217866,2012,7,12.291640,57,11,2,26
4,불량남녀,쇼박스,코미디,108,15세 관람가,신근호,1.00,1,251,2,483387,2010,11,13.088575,110,11,1,282


##**원-핫 인코딩**

In [ ]:
# X, y 분리
X = train.drop(['box_off_num', 'title', 'box_off_num_log'], axis=1)
y = train['box_off_num']

# '기타'를 NaN으로 대체
X = X.replace('기타', np.nan)
test = test.replace('기타', np.nan)

# 3) train과 test를 합쳐서 one-hot 인코딩
full = pd.concat([X, test], axis=0)

# 4) 한 번에 get_dummies
full_encoded = pd.get_dummies(full, drop_first=True)

# 5) 다시 train/test로 나누기
X_encoded = full_encoded.iloc[:len(X), :]
test_encoded = full_encoded.iloc[len(X):, :]

print("X_encoded:", X_encoded.shape)
print("test_encoded:", test_encoded.shape)

X_encoded: (600, 1067)
test_encoded: (243, 1067)


In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#Ridge Regression

In [ ]:
ridge_params = {
    'alpha': [0.1, 1, 10, 100, 1000],
    'solver': ['auto', 'svd', 'cholesky']
}

ridge = Ridge(random_state=42)
ridge_grid = GridSearchCV(
    ridge,
    ridge_params,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("=" * 50)
ridge_grid.fit(X_encoded, y)
print(f"Best params: {ridge_grid.best_params_}")
print(f"Best score: {-ridge_grid.best_score_:.4f}")

Ridge Regression GridSearch 시작...
Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best params: {'alpha': 10, 'solver': 'auto'}
Best score: 2182207079348.9194


#Lasso Regression

In [ ]:
lasso_params = {
    'alpha': [0.001, 0.01, 0.1, 1, 10],
    'max_iter': [1000, 5000]
}

lasso = Lasso(random_state=42)
lasso_grid = GridSearchCV(
    lasso,
    lasso_params,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("\n" + "=" * 50)
lasso_grid.fit(X_encoded, y)
print(f"Best params: {lasso_grid.best_params_}")
print(f"Best score: {-lasso_grid.best_score_:.4f}")


Lasso Regression GridSearch 시작...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best params: {'alpha': 10, 'max_iter': 1000}
Best score: 14677782699224.2148


# ElasticNet


In [ ]:
elasticnet_params = {
    'alpha': [0.001, 0.01, 0.1, 1],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    'max_iter': [1000, 5000]
}

elasticnet = ElasticNet(random_state=42)
elasticnet_grid = GridSearchCV(
    elasticnet,
    elasticnet_params,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("\n" + "=" * 50)
elasticnet_grid.fit(X_encoded, y)
print(f"Best params: {elasticnet_grid.best_params_}")
print(f"Best score: {-elasticnet_grid.best_score_:.4f}")


ElasticNet GridSearch 시작...
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best params: {'alpha': 0.1, 'l1_ratio': 0.9, 'max_iter': 1000}
Best score: 2183064646828.7949


# Random Forest

In [ ]:
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestRegressor(random_state=42)
rf_grid = GridSearchCV(
    rf,
    rf_params,
    cv=3,  # RF는 시간이 오래 걸리므로 cv=3
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("\n" + "=" * 50)
rf_grid.fit(X_encoded, y)
print(f"Best params: {rf_grid.best_params_}")
print(f"Best score: {-rf_grid.best_score_:.4f}")


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best params: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best score: 2048815554921.8738


# Gradient Boosting

In [ ]:
gb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5]
}

gb = GradientBoostingRegressor(random_state=42)
gb_grid = GridSearchCV(
    gb,
    gb_params,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("\n" + "=" * 50)
print("Gradient Boosting GridSearch 시작...")
gb_grid.fit(X_encoded, y)
print(f"Best params: {gb_grid.best_params_}")
print(f"Best score: {-gb_grid.best_score_:.4f}")


Gradient Boosting GridSearch 시작...
Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best params: {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_split': 5, 'n_estimators': 300}
Best score: 2096330673620.1838


# XGBoost

In [ ]:
xgb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb = XGBRegressor(random_state=42, objective='reg:squarederror')
xgb_grid = GridSearchCV(
    xgb,
    xgb_params,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("\n" + "=" * 50)
print("XGBoost GridSearch 시작...")
xgb_grid.fit(X_encoded, y)
print(f"Best params: {xgb_grid.best_params_}")
print(f"Best score: {-xgb_grid.best_score_:.4f}")


XGBoost GridSearch 시작...
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}
Best score: 2175955195221.3333


# LightGBM

In [ ]:
# 컬럼명 정리 함수
def clean_column_names(df):
    """LightGBM이 인식할 수 있도록 컬럼명에서 특수문자 제거"""
    df.columns = df.columns.str.replace('[', '_', regex=False)
    df.columns = df.columns.str.replace(']', '_', regex=False)
    df.columns = df.columns.str.replace('<', '_', regex=False)
    df.columns = df.columns.str.replace('>', '_', regex=False)
    df.columns = df.columns.str.replace('{', '_', regex=False)
    df.columns = df.columns.str.replace('}', '_', regex=False)
    df.columns = df.columns.str.replace(',', '_', regex=False)
    df.columns = df.columns.str.replace(':', '_', regex=False)
    df.columns = df.columns.str.replace('"', '_', regex=False)
    df.columns = df.columns.str.replace("'", '_', regex=False)
    df.columns = df.columns.str.replace(' ', '_', regex=False)
    df.columns = df.columns.str.replace('(', '_', regex=False)
    df.columns = df.columns.str.replace(')', '_', regex=False)
    return df

# 적용
X_encoded = clean_column_names(X_encoded)
test_encoded = clean_column_names(test_encoded)

print("컬럼명 정리 완료!")
print("샘플 컬럼명:", X_encoded.columns[:5].tolist())

컬럼명 정리 완료!
샘플 컬럼명: ['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']


In [ ]:
lgbm_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, -1],
    'num_leaves': [31, 50, 100],
    'subsample': [0.8, 1.0]
}

lgbm = LGBMRegressor(random_state=42, verbose=-1)
lgbm_grid = GridSearchCV(
    lgbm,
    lgbm_params,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("\n" + "=" * 50)
lgbm_grid.fit(X_encoded, y)
print(f"Best params: {lgbm_grid.best_params_}")
print(f"Best score: {-lgbm_grid.best_score_:.4f}")


Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best params: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100, 'num_leaves': 31, 'subsample': 0.8}
Best score: 2103248007118.5273


# 결과 비교

In [ ]:
models_results = {
    'Ridge': ridge_grid,
    'Lasso': lasso_grid,
    'ElasticNet': elasticnet_grid,
    'Random Forest': rf_grid,
    'Gradient Boosting': gb_grid,
    'XGBoost': xgb_grid,
    'LightGBM': lgbm_grid
}

print("모델 성능 비교 (CV MSE)")

results_df = pd.DataFrame({
    'Model': list(models_results.keys()),
    'Best_CV_MSE': [-model.best_score_ for model in models_results.values()],
    'Best_CV_RMSE': [np.sqrt(-model.best_score_) for model in models_results.values()]
})

results_df = results_df.sort_values('Best_CV_RMSE')
print(results_df.to_string(index=False))

모델 성능 비교 (CV MSE)
            Model  Best_CV_MSE  Best_CV_RMSE
    Random Forest 2.048816e+12  1.431368e+06
Gradient Boosting 2.096331e+12  1.447871e+06
         LightGBM 2.103248e+12  1.450258e+06
          XGBoost 2.175955e+12  1.475112e+06
            Ridge 2.182207e+12  1.477230e+06
       ElasticNet 2.183065e+12  1.477520e+06
            Lasso 1.467778e+13  3.831159e+06


# 앙상블

In [ ]:
# 이전에 GridSearch로 찾은 best estimator들
models_dict = {
    'Ridge': ridge_grid.best_estimator_,
    'Lasso': lasso_grid.best_estimator_,
    'ElasticNet': elasticnet_grid.best_estimator_,
    'RF': rf_grid.best_estimator_,
    'GB': gb_grid.best_estimator_,
    'XGB': xgb_grid.best_estimator_,
    'LGBM': lgbm_grid.best_estimator_
}

# Voting Regressor - 다양한 조합

In [ ]:
from itertools import combinations

In [ ]:
voting_results = []

# 2개 모델 조합
print("\n[2개 모델 조합]")
for combo in combinations(models_dict.keys(), 2):
    estimators = [(name, models_dict[name]) for name in combo]
    voting = VotingRegressor(estimators=estimators)

    cv_scores = cross_val_score(
        voting, X_encoded, y,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    mse = -cv_scores.mean()
    rmse = np.sqrt(mse)
    std = cv_scores.std()

    voting_results.append({
        'Type': 'Voting',
        'Models': ' + '.join(combo),
        'N_Models': 2,
        'CV_RMSE': rmse,
        'CV_STD': std
    })

    print(f"{' + '.join(combo):40s} | RMSE: {rmse:.4f} | STD: {std:.4f}")


[2개 모델 조합]
Ridge + Lasso                            | RMSE: 2310593.3722 | STD: 2249668447918.4883
Ridge + ElasticNet                       | RMSE: 1476470.0331 | STD: 971201477610.2401
Ridge + RF                               | RMSE: 1436616.0266 | STD: 983231589783.9482
Ridge + GB                               | RMSE: 1437231.1985 | STD: 912432621913.7229
Ridge + XGB                              | RMSE: 1432223.6100 | STD: 937862193281.0259
Ridge + LGBM                             | RMSE: 1424813.8789 | STD: 944849858461.2220
Lasso + ElasticNet                       | RMSE: 2318516.7693 | STD: 2215048271142.1240
Lasso + RF                               | RMSE: 2298328.5690 | STD: 2218828333124.4414
Lasso + GB                               | RMSE: 2315830.0663 | STD: 2226195542175.8325
Lasso + XGB                              | RMSE: 2306738.9949 | STD: 2203541571672.5869
Lasso + LGBM                             | RMSE: 2287239.6582 | STD: 2105876911685.4248
ElasticNet + RF          

In [ ]:
# 3개 모델 조합
print("\n[3개 모델 조합]")
for combo in combinations(models_dict.keys(), 3):
    estimators = [(name, models_dict[name]) for name in combo]
    voting = VotingRegressor(estimators=estimators)

    cv_scores = cross_val_score(
        voting, X_encoded, y,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    mse = -cv_scores.mean()
    rmse = np.sqrt(mse)
    std = cv_scores.std()

    voting_results.append({
        'Type': 'Voting',
        'Models': ' + '.join(combo),
        'N_Models': 3,
        'CV_RMSE': rmse,
        'CV_STD': std
    })

    print(f"{' + '.join(combo):40s} | RMSE: {rmse:.4f} | STD: {std:.4f}")


[3개 모델 조합]
Ridge + Lasso + ElasticNet               | RMSE: 1901293.3486 | STD: 1128626966275.6011
Ridge + Lasso + RF                       | RMSE: 1877591.8338 | STD: 1121619172155.9580
Ridge + Lasso + GB                       | RMSE: 1886016.3442 | STD: 1086639640485.6868
Ridge + Lasso + XGB                      | RMSE: 1881746.8340 | STD: 1103051861154.1089
Ridge + Lasso + LGBM                     | RMSE: 1869801.9317 | STD: 1074318765086.5897
Ridge + ElasticNet + RF                  | RMSE: 1441338.6002 | STD: 970300462159.8193
Ridge + ElasticNet + GB                  | RMSE: 1440844.0508 | STD: 918801360849.2820
Ridge + ElasticNet + XGB                 | RMSE: 1438970.9940 | STD: 942048603729.7422
Ridge + ElasticNet + LGBM                | RMSE: 1433976.1861 | STD: 947417795779.2097
Ridge + RF + GB                          | RMSE: 1435678.6374 | STD: 945828811484.5492
Ridge + RF + XGB                         | RMSE: 1430723.7560 | STD: 958910403381.5818
Ridge + RF + LGBM         

In [ ]:
# 4개 모델 조합
print("\n[4개 모델 조합]")
for combo in combinations(models_dict.keys(), 4):
    estimators = [(name, models_dict[name]) for name in combo]
    voting = VotingRegressor(estimators=estimators)

    cv_scores = cross_val_score(
        voting, X_encoded, y,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    mse = -cv_scores.mean()
    rmse = np.sqrt(mse)
    std = cv_scores.std()

    voting_results.append({
        'Type': 'Voting',
        'Models': ' + '.join(combo),
        'N_Models': 4,
        'CV_RMSE': rmse,
        'CV_STD': std
    })

    print(f"{' + '.join(combo):40s} | RMSE: {rmse:.4f} | STD: {std:.4f}")


[4개 모델 조합]
Ridge + Lasso + ElasticNet + RF          | RMSE: 1710464.8994 | STD: 859400027336.0049
Ridge + Lasso + ElasticNet + GB          | RMSE: 1715307.9477 | STD: 812392913397.3733
Ridge + Lasso + ElasticNet + XGB         | RMSE: 1712831.2241 | STD: 840604407347.6189
Ridge + Lasso + ElasticNet + LGBM        | RMSE: 1704642.1368 | STD: 836828110097.7286
Ridge + Lasso + RF + GB                  | RMSE: 1706624.6054 | STD: 815252173081.2463
Ridge + Lasso + RF + XGB                 | RMSE: 1702680.8680 | STD: 837778918110.3577
Ridge + Lasso + RF + LGBM                | RMSE: 1691662.6391 | STD: 830922331863.2311
Ridge + Lasso + GB + XGB                 | RMSE: 1708867.1601 | STD: 792185568487.7061
Ridge + Lasso + GB + LGBM                | RMSE: 1696171.8158 | STD: 783472633108.4681
Ridge + Lasso + XGB + LGBM               | RMSE: 1695702.5026 | STD: 810807563056.4863
Ridge + ElasticNet + RF + GB             | RMSE: 1433446.6929 | STD: 939765569583.5609
Ridge + ElasticNet + RF + XGB  

In [ ]:
# 5개 모델 조합
print("\n[5개 모델 조합]")
for combo in combinations(models_dict.keys(), 5):
    estimators = [(name, models_dict[name]) for name in combo]
    voting = VotingRegressor(estimators=estimators)

    cv_scores = cross_val_score(
        voting, X_encoded, y,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    mse = -cv_scores.mean()
    rmse = np.sqrt(mse)
    std = cv_scores.std()

    voting_results.append({
        'Type': 'Voting',
        'Models': ' + '.join(combo),
        'N_Models': 5,
        'CV_RMSE': rmse,
        'CV_STD': std
    })

    print(f"{' + '.join(combo):40s} | RMSE: {rmse:.4f} | STD: {std:.4f}")


[5개 모델 조합]
Ridge + Lasso + ElasticNet + RF + GB     | RMSE: 1619482.2109 | STD: 770283160012.6256
Ridge + Lasso + ElasticNet + RF + XGB    | RMSE: 1616933.7523 | STD: 793455190797.3098
Ridge + Lasso + ElasticNet + RF + LGBM   | RMSE: 1608962.4689 | STD: 795395985518.4404
Ridge + Lasso + ElasticNet + GB + XGB    | RMSE: 1620859.6019 | STD: 750777861714.9607
Ridge + Lasso + ElasticNet + GB + LGBM   | RMSE: 1611750.9373 | STD: 751721313981.1536
Ridge + Lasso + ElasticNet + XGB + LGBM  | RMSE: 1611545.6823 | STD: 778199634799.6584
Ridge + Lasso + RF + GB + XGB            | RMSE: 1618092.8817 | STD: 756573636074.0573
Ridge + Lasso + RF + GB + LGBM           | RMSE: 1607095.2251 | STD: 755830545685.4978
Ridge + Lasso + RF + XGB + LGBM          | RMSE: 1605902.3504 | STD: 778391757396.3523
Ridge + Lasso + GB + XGB + LGBM          | RMSE: 1609593.7227 | STD: 735411432736.7595
Ridge + ElasticNet + RF + GB + XGB       | RMSE: 1430151.7912 | STD: 933814363281.2931
Ridge + ElasticNet + RF + GB + 

In [ ]:
# 전체 모델 조합
print("\n[전체 모델 조합 (7개)]")
estimators = [(name, model) for name, model in models_dict.items()]
voting_all = VotingRegressor(estimators=estimators)

cv_scores = cross_val_score(
    voting_all, X_encoded, y,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

mse = -cv_scores.mean()
rmse = np.sqrt(mse)
std = cv_scores.std()

voting_results.append({
    'Type': 'Voting',
    'Models': 'ALL (7 models)',
    'N_Models': 7,
    'CV_RMSE': rmse,
    'CV_STD': std
})

print(f"{'ALL 7 models':40s} | RMSE: {rmse:.4f} | STD: {std:.4f}")


[전체 모델 조합 (7개)]
ALL 7 models                             | RMSE: 1526364.8628 | STD: 782414023842.2618


# Stacking Regressor - 다양한 조합

In [ ]:
from sklearn.ensemble import StackingRegressor

stacking_results = []

# Meta-learner 옵션
meta_learners = {
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1, max_iter=5000),
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=5000)
}

# 3개 모델 조합 + 다양한 meta-learner
print("\n[3개 모델 조합 with Different Meta-learners]")
for combo in list(combinations(models_dict.keys(), 3))[:10]:  # 상위 10개만
    for meta_name, meta_model in meta_learners.items():
        estimators = [(name, models_dict[name]) for name in combo]

        stacking = StackingRegressor(
            estimators=estimators,
            final_estimator=meta_model,
            cv=5
        )

        cv_scores = cross_val_score(
            stacking, X_encoded, y,
            cv=3,  # Stacking은 시간이 오래 걸려서 cv=3
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )

        mse = -cv_scores.mean()
        rmse = np.sqrt(mse)
        std = cv_scores.std()

        stacking_results.append({
            'Type': f'Stacking-{meta_name}',
            'Models': ' + '.join(combo),
            'N_Models': 3,
            'CV_RMSE': rmse,
            'CV_STD': std
        })

        model_str = ' + '.join(combo)
        print(f"{model_str:40s} | Meta: {meta_name:10s} | RMSE: {rmse:.4f}")


[3개 모델 조합 with Different Meta-learners]
Ridge + Lasso + ElasticNet               | Meta: Ridge      | RMSE: 1502365.2514
Ridge + Lasso + ElasticNet               | Meta: Lasso      | RMSE: 1502365.2514
Ridge + Lasso + ElasticNet               | Meta: ElasticNet | RMSE: 1502365.2514
Ridge + Lasso + RF                       | Meta: Ridge      | RMSE: 1426932.7495
Ridge + Lasso + RF                       | Meta: Lasso      | RMSE: 1426932.7495
Ridge + Lasso + RF                       | Meta: ElasticNet | RMSE: 1426932.7495
Ridge + Lasso + GB                       | Meta: Ridge      | RMSE: 1448193.6574
Ridge + Lasso + GB                       | Meta: Lasso      | RMSE: 1448193.6574
Ridge + Lasso + GB                       | Meta: ElasticNet | RMSE: 1448193.6574
Ridge + Lasso + XGB                      | Meta: Ridge      | RMSE: 1459722.3395
Ridge + Lasso + XGB                      | Meta: Lasso      | RMSE: 1459722.3395
Ridge + Lasso + XGB                      | Meta: ElasticNet | RMSE: 

In [ ]:
# 4개 모델 조합 (상위 성능 base learner 위주)
print("\n[4개 모델 조합 - Top Performers]")
top_4_combos = [
    ['XGB', 'LGBM', 'GB', 'RF'],
    ['XGB', 'LGBM', 'RF', 'Ridge'],
    ['LGBM', 'GB', 'RF', 'Ridge'],
    ['XGB', 'GB', 'RF', 'ElasticNet']
]

for combo in top_4_combos:
    for meta_name, meta_model in meta_learners.items():
        estimators = [(name, models_dict[name]) for name in combo]

        stacking = StackingRegressor(
            estimators=estimators,
            final_estimator=meta_model,
            cv=5
        )

        cv_scores = cross_val_score(
            stacking, X_encoded, y,
            cv=3,
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )

        mse = -cv_scores.mean()
        rmse = np.sqrt(mse)
        std = cv_scores.std()

        stacking_results.append({
            'Type': f'Stacking-{meta_name}',
            'Models': ' + '.join(combo),
            'N_Models': 4,
            'CV_RMSE': rmse,
            'CV_STD': std
        })

        model_str = ' + '.join(combo)
        print(f"{model_str:40s} | Meta: {meta_name:10s} | RMSE: {rmse:.4f}")


[4개 모델 조합 - Top Performers]
XGB + LGBM + GB + RF                     | Meta: Ridge      | RMSE: 1502157.1537
XGB + LGBM + GB + RF                     | Meta: Lasso      | RMSE: 1502157.1537
XGB + LGBM + GB + RF                     | Meta: ElasticNet | RMSE: 1502157.1537
XGB + LGBM + RF + Ridge                  | Meta: Ridge      | RMSE: 1478655.2017
XGB + LGBM + RF + Ridge                  | Meta: Lasso      | RMSE: 1478655.2017
XGB + LGBM + RF + Ridge                  | Meta: ElasticNet | RMSE: 1478655.2017
LGBM + GB + RF + Ridge                   | Meta: Ridge      | RMSE: 1482782.6312
LGBM + GB + RF + Ridge                   | Meta: Lasso      | RMSE: 1482782.6312
LGBM + GB + RF + Ridge                   | Meta: ElasticNet | RMSE: 1482782.6312
XGB + GB + RF + ElasticNet               | Meta: Ridge      | RMSE: 1478396.5593
XGB + GB + RF + ElasticNet               | Meta: Lasso      | RMSE: 1478396.5593
XGB + GB + RF + ElasticNet               | Meta: ElasticNet | RMSE: 1478396.5593

In [ ]:
# 5개 모델 조합
print("\n[5개 모델 조합 - Top Performers]")
top_5_combo = ['XGB', 'LGBM', 'GB', 'RF', 'Ridge']

for meta_name, meta_model in meta_learners.items():
    estimators = [(name, models_dict[name]) for name in top_5_combo]

    stacking = StackingRegressor(
        estimators=estimators,
        final_estimator=meta_model,
        cv=5
    )

    cv_scores = cross_val_score(
        stacking, X_encoded, y,
        cv=3,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    mse = -cv_scores.mean()
    rmse = np.sqrt(mse)
    std = cv_scores.std()

    stacking_results.append({
        'Type': f'Stacking-{meta_name}',
        'Models': ' + '.join(top_5_combo),
        'N_Models': 5,
        'CV_RMSE': rmse,
        'CV_STD': std
    })

    model_str = ' + '.join(top_5_combo)
    print(f"{model_str:40s} | Meta: {meta_name:10s} | RMSE: {rmse:.4f}")


[5개 모델 조합 - Top Performers]
XGB + LGBM + GB + RF + Ridge             | Meta: Ridge      | RMSE: 1502699.7652
XGB + LGBM + GB + RF + Ridge             | Meta: Lasso      | RMSE: 1502699.7652
XGB + LGBM + GB + RF + Ridge             | Meta: ElasticNet | RMSE: 1502699.7652


In [ ]:
print("\n" + "=" * 70)
print("전체 앙상블 결과 정리")
print("=" * 70)

# 단일 모델 결과도 추가
single_results = []
for name, model in models_dict.items():
    cv_scores = cross_val_score(
        model, X_encoded, y,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    mse = -cv_scores.mean()
    rmse = np.sqrt(mse)
    std = cv_scores.std()

    single_results.append({
        'Type': 'Single',
        'Models': name,
        'N_Models': 1,
        'CV_RMSE': rmse,
        'CV_STD': std
    })

# 모든 결과 합치기
all_results = single_results + voting_results + stacking_results
results_df = pd.DataFrame(all_results)

# 정렬 및 출력
results_df = results_df.sort_values('CV_RMSE').reset_index(drop=True)

print("\n[TOP 20 모델/앙상블]")
print(results_df.head(20).to_string(index=False))


전체 앙상블 결과 정리

[TOP 20 모델/앙상블]
  Type                               Models  N_Models      CV_RMSE       CV_STD
Voting               ElasticNet + GB + LGBM         3 1.418018e+06 8.999040e+11
Voting                    Ridge + GB + LGBM         3 1.418493e+06 9.098719e+11
Voting               ElasticNet + RF + LGBM         3 1.418649e+06 9.525990e+11
Voting                    Ridge + RF + LGBM         3 1.419819e+06 9.616379e+11
Voting              ElasticNet + XGB + LGBM         3 1.419876e+06 9.219574e+11
Voting         ElasticNet + RF + XGB + LGBM         4 1.420698e+06 9.424192e+11
Voting                   Ridge + XGB + LGBM         3 1.420913e+06 9.310322e+11
Voting          ElasticNet + RF + GB + LGBM         4 1.421201e+06 9.290592e+11
Voting         ElasticNet + GB + XGB + LGBM         4 1.421494e+06 9.035505e+11
Voting Ridge + ElasticNet + GB + XGB + LGBM         5 1.421545e+06 9.147435e+11
Voting  Ridge + ElasticNet + RF + GB + LGBM         5 1.421559e+06 9.340935e+11
Voting Ri

# 최종 모델 선택 및 예측

In [ ]:
best_row = results_df.iloc[0]
print("\n" + "=" * 70)
print("최고 성능 앙상블")
print("=" * 70)
print(f"Type: {best_row['Type']}")
print(f"Models: {best_row['Models']}")
print(f"CV RMSE: {best_row['CV_RMSE']:.4f}")
print(f"CV STD: {best_row['CV_STD']:.4f}")

# 최고 성능 모델 재학습
best_type = best_row['Type']
best_models = best_row['Models']

if best_type == 'Single':
    final_model = models_dict[best_models]
elif best_type == 'Voting':
    if best_models == 'ALL (7 models)':
        estimators = [(name, model) for name, model in models_dict.items()]
    else:
        model_names = best_models.split(' + ')
        estimators = [(name, models_dict[name]) for name in model_names]
    final_model = VotingRegressor(estimators=estimators)
    final_model.fit(X_encoded, y)
else:  # Stacking
    meta_name = best_type.split('-')[1]
    model_names = best_models.split(' + ')
    estimators = [(name, models_dict[name]) for name in model_names]
    meta_model = meta_learners[meta_name]
    final_model = StackingRegressor(estimators=estimators, final_estimator=meta_model, cv=5)
    final_model.fit(X_encoded, y)

# 테스트 데이터 예측
print("\n테스트 데이터 예측 중...")
test_predictions = final_model.predict(test_encoded)

# 제출 파일 생성
submission = pd.DataFrame({
    'id': range(len(test_predictions)),
    'box_off_num': test_predictions
})

submission.to_csv('submission_best_ensemble.csv', index=False)
print("✅ 예측 완료! submission_best_ensemble.csv 파일로 저장되었습니다.")

# 예측값 통계
print("\n예측값 통계:")
print(f"평균: {test_predictions.mean():.2f}")
print(f"중앙값: {np.median(test_predictions):.2f}")
print(f"최소값: {test_predictions.min():.2f}")
print(f"최대값: {test_predictions.max():.2f}")
print(f"표준편차: {test_predictions.std():.2f}")

# 전체 결과 저장
results_df.to_csv('ensemble_comparison_results.csv', index=False)
print("\n✅ 전체 결과가 ensemble_comparison_results.csv에 저장되었습니다.")

print("\n" + "=" * 70)
print("분석 완료!")
print("=" * 70)


최고 성능 앙상블
Type: Voting
Models: ElasticNet + GB + LGBM
CV RMSE: 1418018.0572
CV STD: 899904024158.7646

테스트 데이터 예측 중...
✅ 예측 완료! submission_best_ensemble.csv 파일로 저장되었습니다.

예측값 통계:
평균: 754139.54
중앙값: 234499.63
최소값: -309542.14
최대값: 9621188.30
표준편차: 1181169.52

✅ 전체 결과가 ensemble_comparison_results.csv에 저장되었습니다.

분석 완료!
